<div style="border:solid green 2px; padding: 20px">
<b>Привет, Глеб!</b>

Меня зовут Александр Пономаренко, и я буду проверять твой проект. Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать data science. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - **пожалуйста, не перемещай, не изменяй и не удаляй их**. Увидев у тебя ошибку, в первый раз я лишь укажу на ее наличие и дам тебе возможность самой найти и исправить ее. На реальной работе твой начальник будет поступать так же, а я пытаюсь подготовить тебя именно к работе аналитиком. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Так выделены небольшие замечания. Я надеюсь, что их ты тоже учтешь - твой проект от этого станет только лучше. Но настаивать на их отработке не буду.

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом.** Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import mean
from pyspark.sql.functions import lit

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        
RANDOM_SEED = 2022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("california - Logistic regression") \
                    .getOrCreate()

In [2]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Огонь, данные на месте:)</div>

In [3]:
print(pd.DataFrame(df.dtypes, columns=['column', 'type']).head(10))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


In [4]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [5]:
columns = df.columns
columns.remove('ocean_proximity')

In [6]:
for i in columns:
    df = df.withColumn(i, 
                       df[i]
                       .cast('float'))

In [7]:
print(pd.DataFrame(df.dtypes, columns=['column', 'type']).head(10))

               column    type
0           longitude   float
1            latitude   float
2  housing_median_age   float
3         total_rooms   float
4      total_bedrooms   float
5          population   float
6          households   float
7       median_income   float
8  median_house_value   float
9     ocean_proximity  string


In [8]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



Пропуски имеются только в 'total_bedrooms', заполним их медианным значением. 

In [9]:
median = df.approxQuantile("total_bedrooms", [0.5], 0.25)[0]
df = df.na.fill({'total_bedrooms':median})

In [10]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Да, хорошо;)</div>

In [11]:
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx']) 
df = indexer.fit(df).transform(df)

In [12]:
encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe'])

In [13]:
df = encoder.fit(df).transform(df)

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, категориальные данные готовы)</div>

Вектор, созданный на основе категориальных признаков у нас только один - ocean_proximity_ohe

Обработаем числовые признаки, используем массив columns, удалив таргет (median_house_value)

In [14]:
columns.remove('median_house_value')

In [15]:
numerical_assembler = VectorAssembler(inputCols=columns,
                                      outputCol="numerical_features")
df = numerical_assembler.transform(df) 

In [16]:
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled")
df = standardScaler.fit(df).transform(df) 

In [17]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'numerical_features', 'numerical_features_scaled']


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отмасштабировал корректно;)</div>

In [18]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
df = final_assembler.transform(df)

df.select(all_features).show(3) 

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.007270679927...|
|      (4,[3],[1.0])|     [-61.002278427488...|
|      (4,[3],[1.0])|     [-61.012259124393...|
+-------------------+-------------------------+
only showing top 3 rows



In [19]:
df.toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,ocean_proximity_ohe,numerical_features,numerical_features_scaled,features
0,-122.230003,37.880001,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","[-122.2300033569336, 37.880001068115234, 41.0,...","[-61.00727067992717, 17.73447826464273, 3.2577...","[0.0, 0.0, 0.0, 1.0, -61.00727067992717, 17.73..."
1,-122.220001,37.860001,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","[-122.22000122070312, 37.86000061035156, 21.0,...","[-61.00227842748807, 17.725114545701565, 1.668...","[0.0, 0.0, 0.0, 1.0, -61.00227842748807, 17.72..."
2,-122.239998,37.849998,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","[-122.23999786376953, 37.849998474121094, 52.0...","[-61.012259124393395, 17.720431793257596, 4.13...","[0.0, 0.0, 0.0, 1.0, -61.012259124393395, 17.7..."
3,-122.250000,37.849998,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","[-122.25, 37.849998474121094, 52.0, 1274.0, 23...","[-61.01725137683249, 17.720431793257596, 4.131...","[0.0, 0.0, 0.0, 1.0, -61.01725137683249, 17.72..."
4,-122.250000,37.849998,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","[-122.25, 37.849998474121094, 52.0, 1627.0, 28...","[-61.01725137683249, 17.720431793257596, 4.131...","[0.0, 0.0, 0.0, 1.0, -61.01725137683249, 17.72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.089996,39.480000,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND,1.0,"(0.0, 1.0, 0.0, 0.0)","[-121.08999633789062, 39.47999954223633, 25.0,...","[-60.43827194902902, 18.48355792046793, 1.9864...","[0.0, 1.0, 0.0, 0.0, -60.43827194902902, 18.48..."
20636,-121.209999,39.490002,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND,1.0,"(0.0, 1.0, 0.0, 0.0)","[-121.20999908447266, 39.4900016784668, 18.0, ...","[-60.498167554379584, 18.4882406729119, 1.4302...","[0.0, 1.0, 0.0, 0.0, -60.498167554379584, 18.4..."
20637,-121.220001,39.430000,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND,1.0,"(0.0, 1.0, 0.0, 0.0)","[-121.22000122070312, 39.43000030517578, 17.0,...","[-60.50315980681869, 18.460149516088414, 1.350...","[0.0, 1.0, 0.0, 0.0, -60.50315980681869, 18.46..."
20638,-121.320000,39.430000,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND,1.0,"(0.0, 1.0, 0.0, 0.0)","[-121.31999969482422, 39.43000030517578, 18.0,...","[-60.55307090729105, 18.460149516088414, 1.430...","[0.0, 1.0, 0.0, 0.0, -60.55307090729105, 18.46..."


# Обучение моделей

In [20]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16418 4222


In [21]:
lr = LinearRegression(labelCol='median_house_value', featuresCol='features')

model = lr.fit(train_data) 

22/11/21 10:02:31 WARN Instrumentation: [4dd82a13] regParam is zero, which might cause numerical instability and overfitting.
22/11/21 10:02:32 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/21 10:02:32 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/11/21 10:02:32 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/21 10:02:32 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
22/11/21 10:02:32 WARN Instrumentation: [4dd82a13] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/11/21 10:02:33 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed


In [22]:
lr_numer = LinearRegression(labelCol='median_house_value', featuresCol='numerical_features_scaled')

model_numer = lr_numer.fit(train_data) 

22/11/21 10:02:36 WARN Instrumentation: [9c81712d] regParam is zero, which might cause numerical instability and overfitting.


# Анализ результатов

Получим предсказания модели, а также создадим константное предсказание

In [23]:
predictions = model.transform(test_data)
predictions_numer = model_numer.transform(test_data)


predictions_constant = predictions.alias('predictions_constant')
constant = predictions_constant.select(mean('median_house_value')).collect()[0][0]
predictions_constant = predictions_constant.withColumn("prediction", lit(constant))

In [25]:
for df, name in zip([predictions, predictions_numer, predictions_constant], ['predictions', 'predictions_numer', 'predictions_constant']):
    print(f'Метрика mse для {name}', RegressionEvaluator(
                                    labelCol="median_house_value", predictionCol="prediction", metricName="rmse").evaluate(df))
    print(f'Метрика rmse для {name}', RegressionEvaluator(
                                    labelCol="median_house_value", predictionCol="prediction", metricName="mse").evaluate(df))
    print(f'Метрика r2 для {name}', RegressionEvaluator(
                                    labelCol="median_house_value", predictionCol="prediction", metricName="r2").evaluate(df))
    print('')

Метрика mse для predictions 68501.14555631217
Метрика rmse для predictions 4692406942.527067
Метрика r2 для predictions 0.6534123623947419

Метрика mse для predictions_numer 69227.01279503122
Метрика rmse для predictions_numer 4792379300.523416
Метрика r2 для predictions_numer 0.6460282663842793

Метрика mse для predictions_constant 116356.67447012724
Метрика rmse для predictions_constant 13538875693.747162
Метрика r2 для predictions_constant -2.220446049250313e-15



<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>  Да, огонь, протестировал корректно:)


</div>

# Вывод

Метрика r2 моделей линейной регрессии отличаются незначительно 0.65 для модели обученной на всех данных и 0.64 для модели обученной только на числовых данных, метрики mse и rmse отличаются также незначительно. 

Также стоит заметить, что прогнозы моделей значительно лучше константных. 

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>  Немного полезного материала:

Немного полезной информации:
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data


</div>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>
<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Глеб, спасибо за хороший проект!!! Я готов принять работу, но хочу убедиться, что тебе все понятно.<br>
Если есть какие либо вопросы я с удовольствием на них отвечу:)</div>


<div class="alert alert-info">

Привет! Спасибо, что проверил работу, у меня вопрос не по проекту, если не ответишь на него - ничего страшного. Ты не знаешь какие-нибудь руководства по catboost (кроме документации)? Там просто много вариантов и нюансов по части настройки, как подбирать гиперпараметры и т.п.

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b>


+  https://habr.com/ru/company/otus/blog/527554/
+  https://www.youtube.com/watch?v=UYDwhuyWYSo&ab_channel=ComputerScienceCenter
+  https://www.youtube.com/watch?v=ZaP5qFSIcIw&t=4398s&ab_channel=Разработка
+  https://dzen.ru/video/watch/62441bf19fcc6f30379205e4
+  https://medium.com/whats-your-data/working-with-categorical-data-catboost-8b5e11267a37
    
Такс, вроде собрал;) Но прям вот прям руководства кроме документации нет. Читаешь, смотришь, решаешь;)
    
    
Удачи в следующем проекте!!!
</div>